# MOUD Types Within 30-Minute Drive (Impedance-Adjusted)

**Created by:** Mallikarjun Siddappa Bhusnoor  
**Last modified:** Jan 22, 2026

### Overview
This notebook takes tract-level access measures for three MOUD provider types — buprenorphine (Bup), methadone (Met), and naltrexone (Nalt) — and identifies where each type is available within a 
30-minute *impedance-adjusted* drive time. 
We first merge the separate Bup/Met/Nalt access tables into a single tract-level table, then create simple 0/1 indicators that flag whether each MOUD type has at least one site within 30 minutes (using the Minutes2 field, which is travel time × 2). 
Finally, we compute a summary variable that counts how many MOUD types are nearby (0–3) for each tract and export the resulting dataset for OEPS integration and mapping.


We’ll use **pandas** for all of the table / DataFrame work in this notebook.  
`import pandas as pd` loads the library and gives it the short alias `pd` so we can write `pd.read_csv(...)`, `pd.merge(...)`, etc.

In [124]:
import pandas as pd

### Load tract-level MOUD access tables

Here we read in the three CSV files that contain tract-level access measures for each MOUD type:

- `Buprenorphine-tract-2020.csv` → buprenorphine measures (`bup`)
- `Methadone-tract-2020.csv` → methadone measures (`met`)
- `Naltrexone-tract-2020_R.csv` → naltrexone measures (`nal`)

Each table has one row per census tract (with GEOID / FIPS / HEROP_ID) and the access metrics we created in the travel-time notebook.

The `bup.head()` call is a quick check to show the first few rows and confirm that the data loaded correctly and the columns look as expected.

In [126]:
bup = pd.read_csv('Buprenorphine-tract-2020.csv')
met = pd.read_csv('Methadone-tract-2020.csv')
nal = pd.read_csv('Naltrexone-tract-2020_R.csv')

In [128]:
bup.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,NAMELSADCO,STATE_NAME,LSAD,HEROP_ID,FIPS,origin,BupCntDr,BupTmDr,BupTmDr2
0,1,89,11021,1400000US01089011021,1089011021,110.21,Census Tract 110.21,AL,Madison County,Alabama,CT,140US01089011021,1089011021,1089011021,6,9.63,19.26
1,1,95,31200,1400000US01095031200,1095031200,312.00,Census Tract 312,AL,Marshall County,Alabama,CT,140US01095031200,1095031200,1095031200,0,88.39,176.78
2,1,73,12401,1400000US01073012401,1073012401,124.01,Census Tract 124.01,AL,Jefferson County,Alabama,CT,140US01073012401,1073012401,1073012401,10,6.70,13.40
3,1,73,3400,1400000US01073003400,1073003400,34.00,Census Tract 34,AL,Jefferson County,Alabama,CT,140US01073003400,1073003400,1073003400,10,6.84,13.68
4,1,73,10402,1400000US01073010402,1073010402,104.02,Census Tract 104.02,AL,Jefferson County,Alabama,CT,140US01073010402,1073010402,1073010402,9,6.92,13.84


In [130]:
met.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,NAMELSADCO,STATE_NAME,LSAD,HEROP_ID,FIPS,origin,MetCntDr,MetTmDr,MetTmDr2
0,1,89,11021,1400000US01089011021,1089011021,110.21,Census Tract 110.21,AL,Madison County,Alabama,CT,140US01089011021,1089011021,1089011021,1,16.04,32.08
1,1,95,31200,1400000US01095031200,1095031200,312.00,Census Tract 312,AL,Marshall County,Alabama,CT,140US01095031200,1095031200,1095031200,0,75.91,151.82
2,1,73,12401,1400000US01073012401,1073012401,124.01,Census Tract 124.01,AL,Jefferson County,Alabama,CT,140US01073012401,1073012401,1073012401,2,19.82,39.64
3,1,73,3400,1400000US01073003400,1073003400,34.00,Census Tract 34,AL,Jefferson County,Alabama,CT,140US01073003400,1073003400,1073003400,2,12.99,25.98
4,1,73,10402,1400000US01073010402,1073010402,104.02,Census Tract 104.02,AL,Jefferson County,Alabama,CT,140US01073010402,1073010402,1073010402,1,15.18,30.36


In [132]:
nal.head()

,Unnamed: 0,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,NAMELSADCO,STATE_NAME,LSAD,HEROP_ID,FIPS,origin,NaltCntDr,NaltTmDr,NaltTmDr2
0,0,1,89,11021,1400000US01089011021,1089011021,110.21,Census Tract 110.21,AL,Madison County,Alabama,CT,140US01089011021,1089011021,1089011021,4,9.63,19.26
1,1,1,95,31200,1400000US01095031200,1095031200,312.00,Census Tract 312,AL,Marshall County,Alabama,CT,140US01095031200,1095031200,1095031200,0,88.97,177.94
2,2,1,73,12401,1400000US01073012401,1073012401,124.01,Census Tract 124.01,AL,Jefferson County,Alabama,CT,140US01073012401,1073012401,1073012401,7,6.70,13.40
3,3,1,73,3400,1400000US01073003400,1073003400,34.00,Census Tract 34,AL,Jefferson County,Alabama,CT,140US01073003400,1073003400,1073003400,7,6.84,13.68
4,4,1,73,10402,1400000US01073010402,1073010402,104.02,Census Tract 104.02,AL,Jefferson County,Alabama,CT,140US01073010402,1073010402,1073010402,7,6.92,13.84


#### Merge the three MOUD tables into one

Now we want a **single table** that has, for each census tract, the access measures for all three MOUD types:

- `bup`  → buprenorphine metrics  
- `met`  → methadone metrics  
- `nal`  → naltrexone metrics  

We merge them together on the shared tract identifier `GEOID`.

- We start from `bup`, then **outer join** `met` on `GEOID`, and then **outer join** `nal` on `GEOID`.
- Using `how='outer'` makes sure we **keep every tract** that appears in any of the three tables, even if one MOUD type is missing there (those cells will just be blank/NA).

The result, `moud_types`, is our combined access table with one row per tract and columns for all three MOUD types.

In [134]:
#Merge all three datasets into one table
moud_types = (
    bup
    .merge(met, on='GEOID', how='outer')
    .merge(nal, on='GEOID', how='outer')
)

In [136]:
moud_types.head()

,STATEFP_x,COUNTYFP_x,TRACTCE_x,AFFGEOID_x,GEOID,NAME_x,NAMELSAD_x,STUSPS_x,NAMELSADCO_x,STATE_NAME_x,...,STUSPS,NAMELSADCO,STATE_NAME,LSAD,HEROP_ID,FIPS,origin,NaltCntDr,NaltTmDr,NaltTmDr2
0,1,1,20100,1400000US01001020100,1001020100,201.00,Census Tract 201,AL,Autauga County,Alabama,...,AL,Autauga County,Alabama,CT,140US01001020100,1001020100,1001020100,1,28.05,56.10
1,1,1,20200,1400000US01001020200,1001020200,202.00,Census Tract 202,AL,Autauga County,Alabama,...,AL,Autauga County,Alabama,CT,140US01001020200,1001020200,1001020200,1,26.62,53.24
2,1,1,20300,1400000US01001020300,1001020300,203.00,Census Tract 203,AL,Autauga County,Alabama,...,AL,Autauga County,Alabama,CT,140US01001020300,1001020300,1001020300,1,24.88,49.76
3,1,1,20400,1400000US01001020400,1001020400,204.00,Census Tract 204,AL,Autauga County,Alabama,...,AL,Autauga County,Alabama,CT,140US01001020400,1001020400,1001020400,1,23.41,46.82
4,1,1,20501,1400000US01001020501,1001020501,205.01,Census Tract 205.01,AL,Autauga County,Alabama,...,AL,Autauga County,Alabama,CT,140US01001020501,1001020501,1001020501,1,23.14,46.28


#### Quick check: what columns do we have after the merge?

Before cleaning up the merged table, we print out the full list of column names,This lets us see:
1. which columns came from each original table (suffixes like _x and _y), and
2. what extra columns I might want to drop or rename. 

In [138]:
moud_types.columns.tolist()

['STATEFP_x',
 'COUNTYFP_x',
 'TRACTCE_x',
 'AFFGEOID_x',
 'GEOID',
 'NAME_x',
 'NAMELSAD_x',
 'STUSPS_x',
 'NAMELSADCO_x',
 'STATE_NAME_x',
 'LSAD_x',
 'HEROP_ID_x',
 'FIPS_x',
 'origin_x',
 'BupCntDr',
 'BupTmDr',
 'BupTmDr2',
 'STATEFP_y',
 'COUNTYFP_y',
 'TRACTCE_y',
 'AFFGEOID_y',
 'NAME_y',
 'NAMELSAD_y',
 'STUSPS_y',
 'NAMELSADCO_y',
 'STATE_NAME_y',
 'LSAD_y',
 'HEROP_ID_y',
 'FIPS_y',
 'origin_y',
 'MetCntDr',
 'MetTmDr',
 'MetTmDr2',
 'Unnamed: 0',
 'STATEFP',
 'COUNTYFP',
 'TRACTCE',
 'AFFGEOID',
 'NAME',
 'NAMELSAD',
 'STUSPS',
 'NAMELSADCO',
 'STATE_NAME',
 'LSAD',
 'HEROP_ID',
 'FIPS',
 'origin',
 'NaltCntDr',
 'NaltTmDr',
 'NaltTmDr2']

#### Keep only the columns we need for MOUD‐types work

The merged table `moud_types` has a lot of columns from all three inputs  
(including temporary duplicates like `STATEFP_x`, `STATEFP_y`, etc.).  
For the “MOUD types within 30 minutes” analysis we only need:

- basic tract geography and IDs (STATEFP, COUNTYFP, TRACTCE, GEOID, HEROP_ID, FIPS, etc.)
- the original count and travel-time measures for each MOUD type  
  (`BupCntDr`, `BupTmDr`, `BupTmDr2`, `MetCntDr`, …, `NaltTmDr2`)
- the `origin` tract ID used to link back to other access tables

In [140]:
# Keeping only the columns we care about
cols_to_keep = [
    'STATEFP', 'COUNTYFP', 'TRACTCE', 'AFFGEOID', 'GEOID',
    'NAME', 'NAMELSAD', 'STUSPS', 'NAMELSADCO', 'STATE_NAME', 'LSAD',
    'HEROP_ID', 'FIPS', 'origin',
    'BupCntDr','BupTmDr', 'BupTmDr2',
    'MetCntDr', 'MetTmDr','MetTmDr2',
    'NaltCntDr', 'NaltTmDr','NaltTmDr2',
]

moud_types_clean = moud_types[cols_to_keep].copy()

In [142]:
moud_types_clean.head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,NAMELSADCO,STATE_NAME,...,origin,BupCntDr,BupTmDr,BupTmDr2,MetCntDr,MetTmDr,MetTmDr2,NaltCntDr,NaltTmDr,NaltTmDr2
0,1,1,20100,1400000US01001020100,1001020100,201.00,Census Tract 201,AL,Autauga County,Alabama,...,1001020100,2,28.05,56.10,1,28.76,57.52,1,28.05,56.10
1,1,1,20200,1400000US01001020200,1001020200,202.00,Census Tract 202,AL,Autauga County,Alabama,...,1001020200,2,26.62,53.24,1,27.34,54.68,1,26.62,53.24
2,1,1,20300,1400000US01001020300,1001020300,203.00,Census Tract 203,AL,Autauga County,Alabama,...,1001020300,2,24.88,49.76,1,25.59,51.18,1,24.88,49.76
3,1,1,20400,1400000US01001020400,1001020400,204.00,Census Tract 204,AL,Autauga County,Alabama,...,1001020400,2,23.41,46.82,1,24.13,48.26,1,23.41,46.82
4,1,1,20501,1400000US01001020501,1001020501,205.01,Census Tract 205.01,AL,Autauga County,Alabama,...,1001020501,2,23.14,46.28,1,23.85,47.70,1,23.14,46.28


### Function to flag “within 30 minutes (impedance time)”

This helper function creates a **yes/no (1/0) indicator** for whether a tract has
access to a MOUD type within 30 minutes, using the impedance-adjusted time.

- **Inputs**
  - `count_col` – column name for the **count of sites within 30 minutes**
  - `time_imp_col` – column name for the **impedance-adjusted travel time**
    (e.g., `Minutes2`)

- **Logic (per tract / per row)**
  - Check **two conditions**:
    - Is the count **greater than 0**?  
      → there is at least one site within the threshold.
    - Is the impedance travel time **less than or equal to 30 minutes**?
  - If **both** conditions are true  
    → return **1** (has access within 30 minutes).
  - Otherwise  
    → return **0** (no access within 30 minutes).

- **Output**
  - A new column of **1s and 0s** (integer values) that we treat as a
    **boolean indicator** for “within 30 minutes (impedance time)”.

In [144]:
def within_30_imp(count_col, time_imp_col):
    return (
        (moud_types_clean[count_col].fillna(0) > 0) &
        (moud_types_clean[time_imp_col] <= 30)
    ).astype(int)

In [146]:
#Use the function for each MOUD type
# These new columns are "flags" (0/1) saying:
# - BupWithin30_imp  → 1 if BUP site within 30 min (imp time), else 0
# - MetWithin30_imp  → same for methadone
# - NaltWithin30_imp → same for naltrexone

moud_types_clean['BupWithin30_imp'] = within_30_imp('BupCntDr', 'BupTmDr2')
moud_types_clean['MetWithin30_imp'] = within_30_imp('MetCntDr', 'MetTmDr2')
moud_types_clean['NaltWithin30_imp'] = within_30_imp('NaltCntDr', 'NaltTmDr2')

In [148]:
moud_types_clean.describe

<bound method NDFrame.describe of        STATEFP  COUNTYFP  TRACTCE              AFFGEOID        GEOID     NAME  \
0            1         1    20100  1400000US01001020100   1001020100   201.00   
1            1         1    20200  1400000US01001020200   1001020200   202.00   
2            1         1    20300  1400000US01001020300   1001020300   203.00   
3            1         1    20400  1400000US01001020400   1001020400   204.00   
4            1         1    20501  1400000US01001020501   1001020501   205.01   
...        ...       ...      ...                   ...          ...      ...   
85182       78        30   960800  1400000US78030960800  78030960800  9608.00   
85183       78        30   960900  1400000US78030960900  78030960900  9609.00   
85184       78        30   961000  1400000US78030961000  78030961000  9610.00   
85185       78        30   961100  1400000US78030961100  78030961100  9611.00   
85186       78        30   961200  1400000US78030961200  78030961200  9612.

### Create `MOUDTypesNearby_imp` – number of MOUD types within 30 minutes

Now that we have a separate **within-30-minute indicator** for each MOUD type
(Bup, Met, Nalt), we want a single variable that tells us **how many different
types** are nearby for each tract.

- **Inputs**
  - `BupWithin30_imp` – 1 if buprenorphine is within 30 min (else 0)
  - `MetWithin30_imp` – 1 if methadone is within 30 min (else 0)
  - `NaltWithin30_imp` – 1 if naltrexone is within 30 min (else 0)

- **Logic**
  - Take these three 0/1 columns and **sum them row-wise** (`axis=1`).
  - Because they are indicators, the sum gives the **count of MOUD types**
    available within 30 minutes for that tract.
  - Possible values:
    - **0** – no MOUD types within 30 minutes  
    - **1** – exactly one type nearby  
    - **2** – two types nearby  
    - **3** – all three types nearby  

- **Example**
  - Suppose a tract has buprenorphine and methadone nearby, but no naltrexone:
    - `BupWithin30_imp = 1`  
    - `MetWithin30_imp = 1`  
    - `NaltWithin30_imp = 0`  
    - `MOUDTypesNearby_imp = 1 + 1 + 0 = 2`  

The resulting `MOUDTypesNearby_imp` column is our **final summary measure** of
how many distinct MOUD options are realistically reachable (within 30 minutes,
using impedance-adjusted travel time) for each tract.

In [150]:
moud_types_clean['MOUDTypesNearby_imp'] = (
    moud_types_clean[['BupWithin30_imp',
                      'MetWithin30_imp',
                      'NaltWithin30_imp']].sum(axis=1)
)

In [152]:
moud_types_clean

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,NAMELSADCO,STATE_NAME,...,MetCntDr,MetTmDr,MetTmDr2,NaltCntDr,NaltTmDr,NaltTmDr2,BupWithin30_imp,MetWithin30_imp,NaltWithin30_imp,MOUDTypesNearby_imp
0,1,1,20100,1400000US01001020100,1001020100,201.00,Census Tract 201,AL,Autauga County,Alabama,...,1,28.76,57.52,1,28.05,56.10,0,0,0,0
1,1,1,20200,1400000US01001020200,1001020200,202.00,Census Tract 202,AL,Autauga County,Alabama,...,1,27.34,54.68,1,26.62,53.24,0,0,0,0
2,1,1,20300,1400000US01001020300,1001020300,203.00,Census Tract 203,AL,Autauga County,Alabama,...,1,25.59,51.18,1,24.88,49.76,0,0,0,0
3,1,1,20400,1400000US01001020400,1001020400,204.00,Census Tract 204,AL,Autauga County,Alabama,...,1,24.13,48.26,1,23.41,46.82,0,0,0,0
4,1,1,20501,1400000US01001020501,1001020501,205.01,Census Tract 205.01,AL,Autauga County,Alabama,...,1,23.85,47.70,1,23.14,46.28,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85182,78,30,960800,1400000US78030960800,78030960800,9608.00,Census Tract 9608,VI,St. Thomas Island,United States Virgin Islands,...,0,NaN,NaN,0,NaN,NaN,0,0,0,0
85183,78,30,960900,1400000US78030960900,78030960900,9609.00,Census Tract 9609,VI,St. Thomas Island,United States Virgin Islands,...,0,NaN,NaN,0,NaN,NaN,0,0,0,0
85184,78,30,961000,1400000US78030961000,78030961000,9610.00,Census Tract 9610,VI,St. Thomas Island,United States Virgin Islands,...,0,NaN,NaN,0,NaN,NaN,0,0,0,0
85185,78,30,961100,1400000US78030961100,78030961100,9611.00,Census Tract 9611,VI,St. Thomas Island,United States Virgin Islands,...,0,NaN,NaN,0,NaN,NaN,0,0,0,0


#### Quick check: tracts with 2 MOUD types nearby

To sanity–check the new `MOUDTypesNearby_imp` variable, we can look at a few
rows where the value equals **2**: (You can change the number to 0,1,3)


In [154]:
moud_types_clean[moud_types_clean['MOUDTypesNearby_imp'] == 2
    ].head()

,STATEFP,COUNTYFP,TRACTCE,AFFGEOID,GEOID,NAME,NAMELSAD,STUSPS,NAMELSADCO,STATE_NAME,...,MetCntDr,MetTmDr,MetTmDr2,NaltCntDr,NaltTmDr,NaltTmDr2,BupWithin30_imp,MetWithin30_imp,NaltWithin30_imp,MOUDTypesNearby_imp
67,1,5,950800,1400000US01005950800,1005950800,9508.0,Census Tract 9508,AL,Barbour County,Alabama,...,0,63.40,126.80,1,0.00,0.00,1,0,1,2
68,1,5,950900,1400000US01005950900,1005950900,9509.0,Census Tract 9509,AL,Barbour County,Alabama,...,0,59.53,119.06,1,14.98,29.96,1,0,1,2
106,1,15,200,1400000US01015000200,1015000200,2.0,Census Tract 2,AL,Calhoun County,Alabama,...,1,10.12,20.24,0,55.72,111.44,1,1,0,2
107,1,15,300,1400000US01015000300,1015000300,3.0,Census Tract 3,AL,Calhoun County,Alabama,...,1,9.70,19.40,0,55.38,110.76,1,1,0,2
108,1,15,400,1400000US01015000400,1015000400,4.0,Census Tract 4,AL,Calhoun County,Alabama,...,1,14.79,29.58,0,57.80,115.60,1,1,0,2


In [156]:
moud_types_clean.describe()

,STATEFP,COUNTYFP,TRACTCE,GEOID,NAME,FIPS,origin,BupCntDr,BupTmDr,BupTmDr2,MetCntDr,MetTmDr,MetTmDr2,NaltCntDr,NaltTmDr,NaltTmDr2,BupWithin30_imp,MetWithin30_imp,NaltWithin30_imp,MOUDTypesNearby_imp
count,85187.000000,85187.000000,85187.000000,8.518700e+04,85187.000000,8.518700e+04,8.518700e+04,85187.000000,82531.000000,82531.000000,85187.000000,78930.000000,78930.000000,85187.000000,82387.000000,82387.000000,85187.000000,85187.000000,85187.000000,85187.000000
mean,28.327679,87.591475,254831.097433,2.841553e+10,2548.310974,2.841553e+10,2.841553e+10,26.372334,11.600954,23.201907,7.284198,18.689847,37.379695,24.124256,11.977540,23.955081,0.742226,0.565943,0.727576,2.035745
std,16.545885,99.610503,348825.638581,1.656990e+10,3488.256386,1.656990e+10,1.656990e+10,41.124703,13.443104,26.886208,11.251038,19.229911,38.459823,37.252586,13.537797,27.075593,0.437412,0.495635,0.445210,1.239764
min,1.000000,1.000000,100.000000,1.001020e+09,1.000000,1.001020e+09,1.001020e+09,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,13.000000,31.000000,10304.000000,1.301596e+10,103.040000,1.301596e+10,1.301596e+10,3.000000,3.510000,7.020000,0.000000,5.870000,11.740000,2.000000,3.660000,7.320000,0.000000,0.000000,0.000000,1.000000
50%,28.000000,65.000000,42500.000000,2.805904e+10,425.000000,2.805904e+10,2.805904e+10,11.000000,6.860000,13.720000,3.000000,11.230000,22.460000,10.000000,7.160000,14.320000,1.000000,1.000000,1.000000,3.000000
75%,42.000000,111.000000,452400.500000,4.201710e+10,4524.005000,4.201710e+10,4.201710e+10,29.000000,14.180000,28.360000,8.000000,23.680000,47.360000,29.000000,14.880000,29.760000,1.000000,1.000000,1.000000,3.000000
max,78.000000,840.000000,991703.000000,7.803096e+10,9917.030000,7.803096e+10,7.803096e+10,227.000000,89.950000,179.900000,53.000000,89.990000,179.980000,206.000000,89.950000,179.900000,1.000000,1.000000,1.000000,3.000000


In [158]:
moud_types_clean.columns.tolist()

['STATEFP',
 'COUNTYFP',
 'TRACTCE',
 'AFFGEOID',
 'GEOID',
 'NAME',
 'NAMELSAD',
 'STUSPS',
 'NAMELSADCO',
 'STATE_NAME',
 'LSAD',
 'HEROP_ID',
 'FIPS',
 'origin',
 'BupCntDr',
 'BupTmDr',
 'BupTmDr2',
 'MetCntDr',
 'MetTmDr',
 'MetTmDr2',
 'NaltCntDr',
 'NaltTmDr',
 'NaltTmDr2',
 'BupWithin30_imp',
 'MetWithin30_imp',
 'NaltWithin30_imp',
 'MOUDTypesNearby_imp']

### Rename columns to final OEPS variable names

Now that we’ve finished all the calculations, we rename the working
columns to their **final** names that will be used in OEPS:

In [160]:
moud_types_clean = moud_types_clean.rename(columns = {
    'BupWithin30_imp': 'BupCntDr2',
    'MetWithin30_imp':'MetCntDr2',
    'NaltWithin30_imp' : 'NaltCntDr2',
    'MOUDTypesNearby_imp':'MoudTyp'
})

### Export final MOUD types table

Finally, we save the cleaned table — with all indicators and the
`MoudTyp` count variable — to a CSV file so it can be used in the
OEPS pipeline and merged with other access outputs.

In [162]:
moud_types_clean.to_csv(
    'BUP_MET_NAL_MOUDTyp.csv',
    index=False
)